In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import os
import pickle

# Cargar el archivo CSV procesado de inmuebles en venta
df = pd.read_csv('inmuebles_venta_procesado_clustering.csv')

# Directorio para guardar los archivos pickle
pickle_dir = 'venta_pickles'
os.makedirs(pickle_dir, exist_ok=True)

# Seleccionar las columnas numéricas que deseas escalar
numeric_cols = ['Precio', 'Superficie Construida', 'Superficie Útil', 'CP_encoded']
numeric_cols = [col for col in numeric_cols if col in df.columns]

# Manejar valores faltantes (rellenar con la mediana)
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

# Instanciar MinMaxScaler para escalar al rango [0, 1]
scaler = MinMaxScaler(feature_range=(0, 1))

# Aplicar el escalado
try:
    df[numeric_cols] = scaler.fit_transform(df[numeric_cols])
    print("Escalado con MinMaxScaler aplicado correctamente.")
except Exception as e:
    print(f"Error al aplicar el escalado: {e}")

# Guardar el objeto escalador en un archivo pickle si no existe
scaler_path = os.path.join(pickle_dir, 'minmax_scaler.pkl')
if not os.path.exists(scaler_path):
    with open(scaler_path, 'wb') as file:
        pickle.dump(scaler, file)
    print(f"Escalador guardado en: {scaler_path}")
else:
    print("El archivo pickle ya existe. No se ha sobrescrito.")

# Guardar el DataFrame procesado y escalado
output_csv_path = 'inmuebles_venta_procesado_escalado.csv'
df.to_csv(output_csv_path, index=False)
print(f"Archivo CSV escalado guardado en: {output_csv_path}")


Escalado con MinMaxScaler aplicado correctamente.
Escalador guardado en: venta_pickles\minmax_scaler.pkl
Archivo CSV escalado guardado en: inmuebles_venta_procesado_escalado.csv


In [ ]:
# Cargar el DataFrame procesado y escalado
df_escalado = pd.read_csv('inmuebles_venta_procesado_escalado.csv')

# Seleccionar las columnas numéricas escaladas
numeric_cols = ['Precio', 'Superficie Construida', 'Superficie Útil', 'CP_encoded']
numeric_cols = [col for col in numeric_cols if col in df_escalado.columns]

# Verificación de escalado con MinMaxScaler
print("\nVerificación de escalado con MinMaxScaler:")

for col in numeric_cols:
    try:
        min_val = df_escalado[col].min()
        max_val = df_escalado[col].max()

        # Verificar si el rango es correcto
        if min_val >= 0 and max_val <= 1:
            print(f"{col} - Escalado correctamente: Mínimo: {min_val:.2f}, Máximo: {max_val:.2f}")
        else:
            print(f"{col} - Problema en el escalado: Mínimo: {min_val:.2f}, Máximo: {max_val:.2f}")
    except Exception as e:
        print(f"Error al verificar la columna {col}: {e}")



Verificación de escalado con MinMaxScaler:
Precio - Escalado correctamente: Mínimo: 0.00, Máximo: 1.00
Superficie Construida - Escalado correctamente: Mínimo: 0.00, Máximo: 1.00
Superficie Útil - Problema en el escalado: Mínimo: 0.00, Máximo: 1.00
CP_encoded - Escalado correctamente: Mínimo: 0.00, Máximo: 1.00


------------------------

# Definir los conjuntos de Train y Test.


In [5]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Cargar el archivo escalado de inmuebles en venta
df = pd.read_csv('inmuebles_venta_procesado_escalado.csv')

# Definir las características (X) y la etiqueta (y)
X = df.drop(columns=['cluster', 'Id']).select_dtypes(include=['float64', 'int64'])  # Solo columnas numéricas
y = df['cluster']  # Etiqueta de los grupos de clustering

# Verificar y manejar posibles valores faltantes (rellenar con la mediana solo para columnas numéricas)
X = X.fillna(X.median(numeric_only=True))

# Dividir los datos en conjuntos de entrenamiento y prueba (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Configurar y entrenar el modelo de clasificación
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = classifier.predict(X_test)

# Guardar el modelo entrenado en un archivo pickle
with open('venta_pickles/random_forest_classifier.pkl', 'wb') as model_file:
    pickle.dump(classifier, model_file)

print("Modelo de clasificación guardado como 'random_forest_classifier.pkl'.")

# Evaluar el modelo
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Modelo de clasificación guardado como 'random_forest_classifier.pkl'.
Accuracy: 0.9935518538420204

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      1403
           1       0.98      0.99      0.99       403
           2       1.00      0.96      0.98        55

    accuracy                           0.99      1861
   macro avg       0.99      0.98      0.99      1861
weighted avg       0.99      0.99      0.99      1861



# Define un modelo de clasificación que pueda clasificar un piso/casa a uno de los grupos creados por el algoritmo de clustering

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import pickle

# Cargar el archivo escalado
df = pd.read_csv('inmuebles_venta_procesado_escalado.csv')

# Verificar las columnas presentes en el DataFrame
print("Columnas del DataFrame:", df.columns)

# Definir las características (X) y la etiqueta (y)
# Asegurarnos de seleccionar solo columnas numéricas
X = df.select_dtypes(include=['float64', 'int64']).drop(columns=['cluster', 'Id'], errors='ignore')
y = df['cluster']

# Verificar las columnas seleccionadas para X
print("Columnas seleccionadas para X (solo numéricas):", X.columns)

# Verificar y manejar posibles valores faltantes
X = X.fillna(X.median(numeric_only=True))

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Configurar y entrenar el modelo de clasificación
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = classifier.predict(X_test)

# Evaluar el modelo
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Guardar el modelo entrenado en un archivo pickle
with open('venta_pickles/modelo_clasificacion_clusters.pkl', 'wb') as model_file:
    pickle.dump(classifier, model_file)

print("Modelo de clasificación guardado como 'modelo_clasificacion_clusters.pkl'.")


Columnas del DataFrame: Index(['Id', 'Descripción', 'Localización', 'Enlace', 'Precio',
       'Última Actualización', 'Timestamp_Scrapeo', 'Superficie Construida',
       'Habitaciones', 'Baños', 'Antigüedad', 'Conservación',
       'Superficie Útil', 'Tipo De Casa', 'CP', 'CP_encoded', 'Planta_11ª',
       'Planta_12ª', 'Planta_13ª', 'Planta_14ª', 'Planta_15ª', 'Planta_16ª',
       'Planta_17ª', 'Planta_1ª', 'Planta_2ª', 'Planta_3ª', 'Planta_4ª',
       'Planta_5ª', 'Planta_6ª', 'Planta_7ª', 'Planta_8ª', 'Planta_9ª',
       'Planta_Bajo', 'Planta_Entresuelo', 'Planta_Más de 20',
       'Planta_No especificado', 'Planta_Principal', 'Planta_Semisótano',
       'Planta_Sótano', 'cluster'],
      dtype='object')
Columnas seleccionadas para X (solo numéricas): Index(['Precio', 'Superficie Construida', 'Antigüedad', 'Conservación',
       'Superficie Útil', 'Tipo De Casa', 'CP', 'CP_encoded', 'Planta_11ª',
       'Planta_12ª', 'Planta_13ª', 'Planta_14ª', 'Planta_15ª', 'Planta_16ª',
       